# Capstone Project
## Finding a possible location and type for a restaurant in Los Angeles County, CA

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>


In this project I will try to give a recommendation on where to open a
restaurant in Los Angeles County, CA. In addition, there shall be given  a
recommendation of which type of restaurant could be opened, based on
existing restaurants in the area and generally popular restaurants in the
whole county.

The decision on where to open a restaurant can be based on many factors,
depending on the target group. For example, one could look for very dense
populated areas, or areas with lots of wealthy citizens.

I will make a decision based on the following conditions:
- Find the area with a good balance between number of possible customers
and a high median income
- the type of restaurant will be determined by the most recommended categories
 of food venue in Los Angeles County, CA and the number of already existing
 venues in
  the area


## Data <a name="data"></a>

I used three different datasets as basis for my analysis. Using these
datasets I am able to work with the following features, among others:
- A list of areas in Los Angeles County, CA based on the ZIP code
- The number of citizens and households in every area
- The estimated median income of every area
- The latitude and longitude for every zip code in the county

<br />

I combined the following datasets for this:
-	2010 Los Angeles Census Data
    - 	https://www.kaggle.com/cityofLA/los-angeles-census-data
-	Median Household Income by Zip Code in 2019
    -	http://www.laalmanac.com/employment/em12c.php
-	US Zip Code Latitude and Longitude
    -   https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/information/

To analyse the existing food venues in the county, the **Foursquare API** is
 used. With this API we can provide the data to answer the following two
 questions:
- What are the most recommended types of restaurants in the county?
- What are the existing food venues categories in the area where we want to
open a restaurant in?